In [284]:
%pylab inline
import pandas as pd
from datetime import datetime

Populating the interactive namespace from numpy and matplotlib


C:\Users\minso\Anaconda3\lib\site-packages\IPython\core\magics\pylab.py:160: UserWarning: pylab import has clobbered these variables: ['table', 'datetime']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [285]:
df = pd.read_excel('SWE sample data.xlsx')
shape(df)

(3553, 4)

### Q1

first subset data for question 1, check the shape

In [267]:
q1df = df[df['country_id']=='BDV']
q1df.shape

(844, 4)

We can find unique 'user_id' with counts for each id using unique function

In [269]:
for i in unique(q1df['site_id']):
    print((i,len(unique(q1df[q1df['site_id']==i]['user_id']))))

('3POLC', 2)
('5NPAU', 544)
('N0OTG', 90)


Using the identified unique site_id, we find length of unique user_id for each unique site_id above shows tuple with site_id as first component and count for unique user in each site_id. Thus we find 5NPAU as the answer with count 544

### Q2

In [273]:
subset = df[(df['ts'] >= '2019-2-3 00:00:00') & (df['ts'] <= '2019-2-4 23:59:59')]
table = subset.groupby(('user_id','site_id')).agg(['count']).iloc[np.where(np.array(subset.groupby(('user_id','site_id')).agg(['count']))[:,0]>=10)]
pd.DataFrame(table[('ts','count')])

C:\Users\minso\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Interpreting tuple 'by' as a list of keys, rather than a single key. Use 'by=[...]' instead of 'by=(...)'. In the future, a tuple will always mean a single key.
  


,,ts
,,count
user_id,site_id,
LC06C3,N0OTG,25
LC3A59,N0OTG,26
LC3C7E,3POLC,15
LC3C9D,N0OTG,17


Using the first Timestamp column, I first subsetted the specified time range in the question.

We can use the groupby function conditioning on two columns, user_id and site_id, and get the count. But since the number of unique user is too big that we can't visually see the count well. Thus we find where(index) in the table the count is over 10, then using the indices create the appropriate table again. Above shows the user_ids and the the site_ids they vitied more than 10 times with the count.

In [274]:
for i in range(0,4):
    (ts,site,count)=pd.DataFrame(table[('ts','count')]).reset_index().iloc[i]
    print((ts,site,count))

('LC06C3', 'N0OTG', 25)
('LC3A59', 'N0OTG', 26)
('LC3C7E', '3POLC', 15)
('LC3C9D', 'N0OTG', 17)


Simplification for the answer as specified in the question. We could just use for loop to check

In [286]:
subset = df[(df['ts'] >= '2019-2-3 00:00:00') & (df['ts'] <= '2019-2-4 23:59:59')]
for i in unique(subset['user_id']):
    s = subset[subset['user_id']==i]
    x = np.array(unique(s['site_id'],return_counts=True))
    for j in range(0,shape(x)[1]):
        if x[1,j] > 10:
            print((i,x[0,j],x[1,j]))

('LC06C3', 'N0OTG', 25)
('LC3A59', 'N0OTG', 26)
('LC3C7E', '3POLC', 15)
('LC3C9D', 'N0OTG', 17)


For each unique user_id i first subsetted the data frame an unique user_id, then found unique site_id with counts. I printed the user_id, site_id, and count if the count is greater than 10

### Q3

In [276]:
subset = df[df['user_id']=='LC3561']
mtx = max(subset['ts'])
(subset[subset['ts']==mtx]['site_id'].tolist()[0]) #just to check understanding of question

'N0OTG'

In [233]:
lastvisit = []
for i in unique(df['user_id']):
    subset = df[df['user_id']==i]
    mtx = max(subset['ts'])
    lastvisit.append(subset[subset['ts']==mtx]['site_id'].tolist()[0])

Here, for each unique user_id, I found the last visited sites in the original data frame. Then created the lists of the last visits.

I first subsetted the dataframe for each unique user id, found maximum time stamp(most recent visit), then appended the site_id that had the corresponding maximum to list 'lastvisit'

In [234]:
unique(lastvisit,return_counts=True)

(array(['3POLC', '5NPAU', 'EUZ/Q', 'GVOFK', 'JSUUP', 'N0OTG', 'QGO3G',
        'RT9Z6'], dtype='<U5'),
 array([ 28, 992,   1,  42,   1, 561, 289,   2], dtype=int64))

Using The unique function, we can see the site_id and corresponding number of unique_user.

In [235]:
countlist = []
counts = np.array(unique(lastvisit,return_counts=True))
for i in range(0,shape(counts)[1]):
    countlist.append((counts[0,i],counts[1,i]))

In [236]:
sorted(countlist,key=lambda tup:tup[1],reverse=True)

[('5NPAU', '992'),
 ('N0OTG', '561'),
 ('GVOFK', '42'),
 ('QGO3G', '289'),
 ('3POLC', '28'),
 ('RT9Z6', '2'),
 ('EUZ/Q', '1'),
 ('JSUUP', '1')]

Then changed the format into tuple and sorted the tuple to get the top 3 pairs, ('5NPAU', '992'), ('N0OTG', '561'), ('GVOFK', '42')

### Q4

We first want to find user_id with count greater than 2, to identify the user who has initial visit and the last visit. User count less than 2 indicate the user visited the site then just left in our given dataset, thus we have first visit but wouldn't have the last visit

But first we can just naively treat the user with count 1 to be included and try, using the last code, but this time we find minimum time stamp (first visit)

In [278]:
firstvisit = []
for i in unique(df['user_id']):
    subset = df[df['user_id']==i]
    mintx = min(subset['ts'])
    firstvisit.append(subset[subset['ts']==mintx]['site_id'].tolist()[0])

In [279]:
c = []
for i in range(0,len(firstvisit)):
    c.append((firstvisit[i]==lastvisit[i]))
sum(c)

1670

Then the number is 1670. Now we try with only users who visited more than once:

In [256]:
u = np.array(unique(df['user_id'],return_counts=True))
freuser = list(u[0,(u[1]>1)])

Here we identify the unique users who appears at least twice

In [260]:
firstvisit2 = []
lastvisit2 = []
for i in freuser:
    subset = df[df['user_id']==i]
    mintx = min(subset['ts'])
    firstvisit2.append(subset[subset['ts']==mintx]['site_id'].tolist()[0])
    mtx = max(subset['ts'])
    lastvisit2.append(subset[subset['ts']==mtx]['site_id'].tolist()[0])

use same code to get last and first visit

In [263]:
c2 = []
for i in range(0,len(firstvisit2)):
    c2.append((firstvisit2[i]==lastvisit2[i]))
sum(c2)

409

Sum up the users who has initial and last visit has equal site_id. When we only count the users who visited site more than once we get 409 as answer